In [ ]:
import pandas as pd 

In [ ]:
df_Details=pd.read_csv('Details.csv')
df_Orders=pd.read_csv('Orders.csv')

In [ ]:
df_Details.head()

,Order ID,Total Sales,Profit,Quantity,Category,Sub-Category,PaymentMode
0,B-25681,1096,658,7,Electronics,Electronic Games,COD
1,B-26055,5729,64,14,Furniture,Chairs,EMI
2,B-25955,2927,146,8,Furniture,Bookcases,EMI
3,B-26093,2847,712,8,Electronics,Printers,Credit Card
4,B-25602,2617,1151,4,Electronics,Phones,Credit Card


In [ ]:
df_Details.isna().sum()

Order ID        0
Total Sales     0
Profit          0
Quantity        0
Category        0
Sub-Category    0
PaymentMode     0
dtype: int64

In [ ]:
df_Details.shape

(1500, 7)

In [ ]:
df_Orders.head()

,Order ID,Order Date,CustomerName,State,City
0,B-26055,10-03-2018,Harivansh,Uttar Pradesh,Mathura
1,B-25993,03-02-2018,Madhav,Delhi,Delhi
2,B-25973,24-01-2018,Madan Mohan,Uttar Pradesh,Mathura
3,B-25923,27-12-2018,Gopal,Maharashtra,Mumbai
4,B-25757,21-08-2018,Vishakha,Madhya Pradesh,Indore


In [ ]:
df_Orders.isna()

,Order ID,Order Date,CustomerName,State,City
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
495,False,False,False,False,False
496,False,False,False,False,False
497,False,False,False,False,False
498,False,False,False,False,False


In [ ]:
df_Orders.columns

Index(['Order ID', 'Order Date', 'CustomerName', 'State', 'City'], dtype='object')

In [ ]:
df_Details.columns

Index(['Order ID', 'Total Sales', 'Profit', 'Quantity', 'Category',
       'Sub-Category', 'PaymentMode'],
      dtype='object')

In [ ]:
# Função simples para limpar os nomes
def padronizar_colunas(df):
    df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
    return df

In [ ]:
# Aplicar nas duas tabelas
df_orders = padronizar_colunas(df_Orders)
df_details = padronizar_colunas(df_Details)

In [ ]:
# Verificar como ficaram
print("Colunas Orders:", df_Orders.columns.tolist())
print("Colunas Details:", df_Details.columns.tolist())

Colunas Orders: ['order_id', 'order_date', 'customername', 'state', 'city']
Colunas Details: ['order_id', 'total_sales', 'profit', 'quantity', 'category', 'sub_category', 'paymentmode']


In [ ]:
df_Details.dtypes

order_id        object
total_sales      int64
profit           int64
quantity         int64
category        object
sub_category    object
paymentmode     object
dtype: object

In [ ]:
df_Orders.dtypes

order_id        object
order_date      object
customername    object
state           object
city            object
dtype: object

In [ ]:
# 1. Limpeza de espaços em branco em todas as colunas de texto
df_orders = df_orders.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df_details = df_details.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
# 2. Conversão de DATA (Essencial para o Power BI)
# O dayfirst=True é importante se a tua data estiver no formato DD-MM-AAAA
df_orders['order_date'] = pd.to_datetime(df_orders['order_date'], dayfirst=True)

In [ ]:
# 3. Garantir que IDs são Strings puras para o cruzamento
df_orders['order_id'] = df_orders['order_id'].astype(str)
df_details['order_id'] = df_details['order_id'].astype(str)

In [ ]:
print("✅ Tipos corrigidos e espaços removidos!")
print(df_orders.dtypes) # Agora order_date deve aparecer como datetime64

✅ Tipos corrigidos e espaços removidos!
order_id                object
order_date      datetime64[ns]
customername            object
state                   object
city                    object
dtype: object


In [ ]:
df_final = pd.merge(df_details, df_orders, on='order_id', how='inner')

print(f"Volume após limpeza: {df_final.shape}")

Volume após limpeza: (1500, 11)


In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Configurações da tua nova base
USUARIO = 'postgres'
SENHA = '12345'
HOST = 'localhost'
PORTA = '5432'
DB_NOME = 'ecommerce_vendas'

# Criar o motor de conexão
engine = create_engine(f'postgresql://{USUARIO}:{SENHA}@{HOST}:{PORTA}/{DB_NOME}')

print(f"✅ Conexão preparada para o banco: {DB_NOME}")

✅ Conexão preparada para o banco: ecommerce_vendas


In [ ]:
# Enviar o DataFrame df_final para a tabela principal
# Como o banco é novo, o if_exists='replace' vai criar a tabela do zero
df_final.to_sql('fato_vendas_consolidada', con=engine, index=False, if_exists='replace')

print("🚀 Sucesso total! Os dados estão agora no banco 'ecommerce_vendas'.")

🚀 Sucesso total! Os dados estão agora no banco 'ecommerce_vendas'.
